In [1]:
import os
import pandas as pd

In [2]:
#read data from cleanup
summarized_offensive_stats = pd.read_csv('../Resources/total_offensive_stats.csv')

In [3]:
#drop unnecessary index column
summarized_offensive_stats = summarized_offensive_stats.drop(columns='Unnamed: 0')

In [4]:
#convert age from string format to numberic & convert to total age in days
summarized_offensive_stats['age_year'] = pd.to_numeric(summarized_offensive_stats['age'].str.split('-').str[0])
summarized_offensive_stats['age_year'] = summarized_offensive_stats['age_year'].multiply(365.25)
summarized_offensive_stats['age_days'] = pd.to_numeric(summarized_offensive_stats['age'].str.split('-').str[1])
summarized_offensive_stats['player_age'] = summarized_offensive_stats['age_year'] + summarized_offensive_stats['age_days']

In [5]:
#drop columns that are not needed for summarizing data
summarized_offensive_stats = summarized_offensive_stats.drop(columns=['date','game_number','opponent','game_won','player_team_score','opponent_score','kick_return_attempts','punt_return_attempts','team'])

In [6]:
#summarize data
summarized_offensive_stats = summarized_offensive_stats.groupby(
    ['player_id','year'], as_index = False
    ).agg(
        {
            'player_age':'mean',
            'passing_attempts':sum,
            'passing_completions':sum,
            'passing_yards':sum,
            'passing_rating':'mean',
            'passing_touchdowns':sum,
            'passing_interceptions':sum,
            'passing_sacks':sum,
            'passing_sacks_yards_lost':sum,
            'rushing_attempts':sum,
            'rushing_yards':sum,
            'rushing_touchdowns':sum,
            'receiving_targets':sum,
            'receiving_receptions':sum,
            'receiving_yards':sum,
            'receiving_touchdowns':sum,
            'kick_return_yards':sum,
            'kick_return_touchdowns':sum,
            'punt_return_yards':sum,
            'punt_return_touchdowns':sum,
            'total_passing_fantasy_points':sum,
            'total_rushing_fantasy_points':sum,
            'total_receiving_fantasy_points':sum,
            'total_other_fantasy_points':sum,
            'total_combined_fantasy_points':sum
        }
    )

In [7]:
#output to csv
summarized_offensive_stats.to_csv('../Resources/summarized_offensive_stats_no_game_location.csv')

In [8]:
#combine all data into one DatFrame

stats_total=pd.read_csv('../Resources/summarized_offensive_stats_no_game_location.csv')
players_total=pd.read_csv('../Resources/offensive_players.csv')


In [9]:
#combined dataframe

combined_df=pd.merge(players_total,stats_total, on='player_id')


In [10]:
#change height into inches
combined_df['height_inches'] = pd.to_numeric(combined_df['height'].str.split('-').str[0])
combined_df['height_inches'] = combined_df['height_inches'].multiply(12)+ \
pd.to_numeric(combined_df['height'].str.split('-').str[1])


In [11]:
position_data = combined_df["position"]
position_cleanup = []
for x in position_data:
    position_cleanup.append(x[:2])
combined_df["position_cleaned"] = position_cleanup

In [12]:
offensive_positions =['QB','RB','TB','HB','WR','TE','FB','WB','SE','FL']
offensive_position_players = combined_df[combined_df["position_cleaned"].isin(offensive_positions)]
combined_df.loc[combined_df["position_cleaned"] == "TB", "position_cleaned"] = "RB"
combined_df.loc[combined_df["position_cleaned"] == "HB", "position_cleaned"] = "RB"
combined_df.loc[combined_df["position_cleaned"] == "FB", "position_cleaned"] = "RB"
combined_df.loc[combined_df["position_cleaned"] == "WB", "position_cleaned"] = "RB"
combined_df.loc[combined_df["position_cleaned"] == "SE", "position_cleaned"] = "WR"
combined_df.loc[combined_df["position_cleaned"] == "FL", "position_cleaned"] = "WR"

In [13]:
combined_df['position_cleaned'].unique()

array(['QB', 'TE', 'RB', 'WR'], dtype=object)

In [14]:
combined_df.to_csv('../Resources/total_combined_data_no_game_location.csv')